### Rechunking Experiments

Test to see if very wide chunks can be rechunked to very tall chunks without OOM.

In [1]:
import os
import numpy as np
import xarray as xr
import pandas as pd
import dask
from sgkit_bgen import read_bgen
from bokeh.io import output_notebook, export_png
from dask.diagnostics import ResourceProfiler, ProgressBar, Profiler, visualize
%load_ext autoreload
%autoreload 2
output_notebook()

Loading BokehJS ...

In [2]:
ds = read_bgen('/home/eczech/data/rs-ukb-local/bgen/ukb_imp_chrXY_v3.bgen', chunks=(10, -1, -1))
ds

,Array,Chunk
Bytes,89.32 GB,19.46 MB
Shape,"(45906, 486443)","(10, 486443)"
Count,22956 Tasks,4591 Chunks
Type,float32,numpy.ndarray


In [3]:
from rechunker import rechunk

In [14]:
!rm -rf /tmp/rechunker.zarr
!rm -rf /tmp/rechunker-tmp.zarr

In [15]:
%%time
# Export sample WITHOUT GP compression filter (peaking at about 8G of 14G used on 4 workers w/ 256 MiB chunks)
with ProgressBar(), ResourceProfiler() as prof:
    arr = rechunk(
        ds['call_genotype_probability'].data[:10000], 
        target_chunks=(1000, 1000, 3), 
        max_mem='2G', 
        target_store='/tmp/rechunker.zarr',
        temp_store='/tmp/rechunker-tmp.zarr'
    ).execute()

[########################################] | 100% Completed |  5min 23.0s
CPU times: user 10min 39s, sys: 56.6 s, total: 11min 36s
Wall time: 5min 23s


In [16]:
!du -ch /tmp/rechunker-tmp.zarr

7.5G	/tmp/rechunker-tmp.zarr
7.5G	total


In [17]:
xr.open_zarr('/tmp/rechunker-tmp.zarr')

ValueError: path '' contains an array